In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
with open('urls.json') as fopen:
    urls = sorted(json.load(fopen))

In [3]:
len(urls)

10480

In [4]:
headers = {
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Google Chrome";v="116"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

In [15]:
'https://www.maukerja.my' + urls[0]

'https://www.maukerja.my/career-advice/blog/20663/Dibuang-Kerja-24-Jam-Siapa-Sangka-Buih-Sabun-Kanak-Kanak-Jadi-Punca-Rezeki-Modal-RM300'

In [9]:
r = requests.get('https://www.maukerja.my' + urls[0], headers = headers)

In [11]:
soup = BeautifulSoup(r.content, "lxml")

In [13]:
soup.find('h1').text

"Dibuang Kerja 24 Jam, Siapa Sangka 'Buih Sabun Kanak-Kanak' Jadi Punca Rezeki, Modal RM300!"

In [16]:
soup.find('div', {'class': 'card-content has-padding-15'}).get_text(separator="\n")

"# Inspirasi & Kerjaya (Mau Sembang) \n \nDibuang Kerja 24 Jam, Siapa Sangka 'Buih Sabun Kanak-Kanak' Jadi Punca Rezeki, Modal RM300!\n \n \nby \nHaziqah Muhamad\n \n\nApr 8, 2019\n\n\npada 6:47 PM\n\n \nMelayari kehidupan kadang kala ada jatuh dan naiknya. Begitulah yang berlaku pada pasangan ini yang dahulunya miliki kerjaya yang begitu kukuh. Keduanya dalam jurusan kejuruteraan dan boleh dikira cemerlang ketika di universiti dulu. Kami temubual pasangan Fariz dan Nashwa yang pernah alami pengalaman di buang kerja dan kepayahan untuk menempatkan diri dalam dunia kerjaya.\n\n\n \n\xa0\n\n\nTONTON VIDEO INI >>>\xa0\nGAJI SEORANG PENJUAL BUBBLE\n\n\n \n\xa0\n\n\nDIBUANG KERJA 24 JAM\n\n\n \n“Saya dulunya bekerja dalam bidang kejuruteraan. Boleh dikatakan kehidupan ketika itu memang senang. Gaji yang kami kumpul sebulan bersama dilihat satu angka yang boleh kami sifatkan sebagai ‘selesa’. Tapi siapa sangka perjalanan itu di uji apabila saya di minta untuk berhenti dan notis yang di kelua

In [17]:
!mkdir page

In [18]:
for no, url in tqdm(enumerate(urls)):
    filename = os.path.join('page', f'{no}.json')
    if os.path.exists(filename):
        continue
    
    success = False
    while True:
        try:
            r = requests.get('https://www.maukerja.my' + url, headers = headers)
            success = True
            break
        except Exception as e:
            if 'xceeded 30 redirects' in str(e):
                break
            time.sleep(1.0)
            
    if not success:
        continue
            
    soup = BeautifulSoup(r.content, "lxml")

    try:
        title = soup.find('h1').text
    except Exception as e:
        title = None

    try:
        body = soup.find('div', {'class': 'card-content has-padding-15'}).get_text(separator="\n")
    except Exception as e:
        body = None

    data = {
        'url': url,
        'title': title,
        'body': body,
    }
    with open(filename, 'w') as fopen:
        json.dump(data, fopen)

10480it [1:03:46,  2.74it/s]


In [19]:
from glob import glob

files = glob('page/*.json')
len(files)

10480

In [20]:
with open('maukerja.my.jsonl', 'w') as fopen_l:
    for f in files:
        with open(f) as fopen:
            d = json.load(fopen)
        fopen_l.write(f'{json.dumps(d)}\n')